In [ ]:
import json
from py2neo import Graph, Node, Relationship

# Need to get authentication working, currently NEO4J_AUTH=none
graph = Graph("bolt://neo4j:7687")
# graph = Graph('bolt://localhost:7687', bolt=True)

#graph.delete_all()

n_nodes = graph.database.primitive_counts['NumberOfNodeIdsInUse']
n_relationships = graph.database.primitive_counts['NumberOfRelationshipIdsInUse']
print("Connected to graph database with {:,} nodes and {:,} relationships!".format(n_nodes, n_relationships))


In [ ]:
import pandas as pd
import time

start_time = time.time()
start_year, end_year, step = 1900, 1900, 1
dfs = []
for year in range(start_year, end_year+1, step):
    
    # < IS MUCH FASTER THAN <=
    print("Running PageRank on patents from < {}...".format(year), end=" ")
    query_start_time = time.time()
    query = """
    CALL algo.pageRank.stream(
         'MATCH (p:Patent) WHERE p.pub_date < "{}-01-01" AND p.pub_date <> "" RETURN id(p) as id'
        ,'MATCH (p1:Patent)-[:CITES]->(p2:Patent) RETURN id(p1) as source, id(p2) as target'
        ,{{graph:'cypher', iterations:20, write:false}})
    YIELD node, score
    WITH * 
    ORDER BY score DESC
    RETURN 
        node.number AS number, 
        score;
    """.format(year)
#     print(query)
    df = graph.run(query).to_data_frame()
    df['year'] = year
    dfs.append(df)
    query_end_time = time.time()
    print("Done ({:.2f} minutes).".format((query_end_time-query_start_time)/60))
    
end_time = time.time()
print("Finished all calculations in {:.2f} minutes.".format((end_time-start_time)/60))

In [ ]:
# import pandas as pd
# import time

# start_time = time.time()
# start_year, end_year, step = 1900, 1905, 5
# dfs = []
# for year in range(start_year, end_year+1, step):
    
#     # < IS MUCH FASTER THAN <=
#     print("Running PageRank on patents from < {}...".format(year), end=" ")
#     query_start_time = time.time()
#     query = """
#     CALL algo.pageRank.stream(
#          'MATCH (p:Patent) WHERE p.pub_date < "{}-01-01" AND p.pub_date <> "" RETURN id(p) as id'
#         ,'MATCH (p1:Patent)-[:CITES]->(p2:Patent) RETURN id(p1) as source, id(p2) as target'
#         ,{{graph:'cypher', iterations:20, write:false}})
#     YIELD node, score
#     WITH * 
#     ORDER BY score DESC

#     RETURN 
#         node.number AS number, 
#         node.title AS title, 
#         node.inventors AS inventors, 
#         node.location AS location, 
#         node.n_citations AS n_citations, 
#         node.pub_date AS pub_date, 
#         node.patent_citations AS patent_citations, 
#         node.classifications AS classifications, 
#         node.cited_by AS cited_by, 
#         score;
#     """.format(year)
# #     print(query)
#     df = graph.run(query).to_data_frame()
#     df['year'] = year
#     dfs.append(df)
#     query_end_time = time.time()
#     print("Done ({:.2f} minutes).".format((query_end_time-query_start_time)/60))
    
# end_time = time.time()
# print("Finished all calculations in {:.2f} minutes.".format((end_time-start_time)/60))

In [ ]:
df

In [ ]:
pubd_list = df['pub_date'].tolist()
print(len(pubd_list[0]))

In [ ]:
print(type(df['score']))
score_list = df['score'].tolist()

print(len(score_list))


In [ ]:
# log scala
import numpy as np
score_list_log = np.log(score_list)

In [ ]:
import seaborn as sns
from scipy import stats, integrate
import matplotlib.pyplot as plt
sns.set()
sns.distplot(score_list);


In [ ]:
import seaborn as sns
sns.set()
#sns.distplot(data_test, rug=True, norm_hist=False)
sns.distplot(score_list,  rug=True, kde=True, hist=False, bins = 100)
sns.distplot(score_list,  rug=True, kde=False, hist=True, norm_hist=False)

In [ ]:
print(score_list[5910160])

In [ ]:
import numpy as np
x1 = np.random.normal(0, 0.8, 10000)
x2 = np.random.normal(-2, 1, 1000)
x3 = np.random.normal(3, 2, 1000)
x1 = [1,2,3,2,4,6,1,2,3]
x2 = [3,5,6,3,4,5,9,5,3,2]
x3 = [6,7,9,8,6,8,9,1]
kwargs = dict(histtype='stepfilled', alpha=0.3, bins=10)

plt.hist(x1, **kwargs)
plt.hist(x2, **kwargs)
plt.hist(x3, **kwargs);

In [ ]:
#print(score_list)
from matplotlib import pyplot as plt
#plt.xlim(0, 1)

plt.hist(score_list, bins = 100)
#plt.hist(data_test, bins = 100)

plt.title('for the year')
plt.xlabel('Pagerank score')
plt.ylabel('count')

plt.show()

In [ ]:
lowest_score = min(score_list)
score_list_wo_lowest = [] 
for i in score_list:
    if i != lowest_score:
        score_list_wo_lowest.append(i)
        
        
score_list_wo_lowest_log = np.log(score_list_wo_lowest)  

In [ ]:
print(len(score_list_wo_lowest))

In [ ]:
#print(score_list)
from matplotlib import pyplot as plt
#plt.xlim(0, 1)


fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.set_facecolor((1, 1, 1))

plt.axis([-2,4,0,250000])
plt.hist(score_list_wo_lowest_log, bins = 200)
#plt.hist(data_test, bins = 100)
font = {'family':'sans-serif',
        'color':  'grey',
        'weight': 'normal',
        'size': 60,
        }
plt.xlabel('Impact Metric')
plt.ylabel('Count')
plt.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))
fontdict=font
plt.text(1, 25000, '2000', fontdict=font)
plt.show()


fig.savefig('Figures/2000.png')


In [ ]:
backup = score_list_wo_lowest_log

In [ ]:
from itertools import groupby
element_count = [len(list(group)) for key, group in groupby(score_list_log)]
print(max(element_count))

In [ ]:
sorted(element_count, reverse=True)[:10]